Forms:
    10-K
    10-Q
    8-K
    Proxy Statement
    Forms 3, 4, 5
    Schedulem13D
    144

10-K:
    NLP:
        Business?
        Risk Factors
        Unresolved Staff Comments
        Properties
        Legal Proceedings
        Mine Safety Disclosures
        

In [1]:
import pandas as pd
import re
import requests
import unicodedata
from bs4 import BeautifulSoup



## Get txt document from SEC website

In [2]:
htmlText = r"https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847.txt"



#get response
response = requests.get(htmlText)

# parse response
soup = BeautifulSoup(response.content, 'lxml')

## Decode Text

In [ ]:
from unicodedata import normalize

print('%r' % normalize('NFD', u'\u00C7'))  # decompose: convert Ç to "C + ̧"
print('%r' % normalize('NFC', u'C\u0327')) # compose: convert "C + ̧" to Ç

In [ ]:
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)




# def remove_newlines(string):
#     # \p, \xa0

## Split up into Parts 1 - 4

PART I

Business 
Risk Factors
Unresolved Staff Comments
Properties
Legal Proceedings
Reserved  

PART II

Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 
Selected Financial Data
Management’s Discussion and Analysis of Financial Condition and Results of Operations
Quantitative and Qualitative Disclosures About Market Risk
Financial Statements and Supplementary Data
Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Controls and Procedures 
Other Information

PART III

Directors, Executive Officers and Corporate Governance
Executive Compensation
Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
Certain Relationships and Related Transactions, and Director Independence
Principal Accountant Fees and Services

Exhibits and Financial Statement Schedules

In [3]:
#initialize master 10-k dict
tenK_document_dict = {}

tenK_text = soup.find('text').extract()

parts_to_split = ['PART I ',
                  'PART II\n',
                  'PART III ',
                 ]

#get different parts of document: I-IV
text_split_into_parts = []

# for parts I-IV
for part in parts_to_split:
    next_part = tenK_text.find('b', text=part)
    print(next_part)
    text_split_into_parts.append(next_part)

    
#convert all parts to string
all_parts = [str(part) for part in text_split_into_parts]

#prep the document text for splitting - convert to string
tenK_string = str(tenK_text)

#defing the regex delimeter pattern
regex_delimiter_pattern = '|'.join(map(re.escape, all_parts))

#split doc on each break
split_tenK_string = re.split(regex_delimiter_pattern, tenK_string)

#store parts in master dict
tenK_document_dict['Part I'] = split_tenK_string[1]
tenK_document_dict['Part II'] = split_tenK_string[2]
tenK_document_dict['Part III'] = split_tenK_string[3]

<b>PART I </b>
<b>PART II
</b>
<b>PART III </b>


In [8]:
len(split_tenK_string)

4

In [9]:
tenK_document_dict['Part III']

'</font></p>\n<p style="font-size:12px;margin-top:0px;margin-bottom:0px">\xa0</p>\n<table border="0" cellpadding="0" cellspacing="0" style="BORDER-COLLAPSE:COLLAPSE" width="100%">\n<tr>\n<td align="left" valign="top" width="10%"><font size="2" style="font-family:Times New Roman"><b><a name="tx151489_15"></a>ITEM\xa010.</b></font></td>\n<td align="left" valign="top"><font size="2" style="font-family:Times New Roman"><b>DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE </b></font></td></tr></table> <p style="margin-top:6px;margin-bottom:0px; text-indent:4%"><font size="2" style="font-family:Times New Roman">The information required by this Item\xa010 of Form 10-K that is found in our 2011 Proxy Statement to be filed with the SEC in connection with the solicitation of proxies for our 2011 Annual Meeting of\nStockholders (2011 Proxy Statement) is incorporated by reference to our 2011 Proxy Statement. The 2011 Proxy Statement will be filed with the SEC within 120 days after the end of 

## Reading tables